## 基于Embedding向量进行文本聚类


In [3]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

def twenty_newsgroup_to_csv():
    newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

    df = pd.DataFrame([newsgroups_train.data, newsgroups_train.target.tolist()]).T
    df.columns = ['text', 'target']

    targets = pd.DataFrame( newsgroups_train.target_names, columns=['title'])

    out = pd.merge(df, targets, left_on='target', right_index=True)
    out.to_csv('20_newsgroup.csv', index=False)
    
twenty_newsgroup_to_csv()


In [24]:
from openai.embeddings_utils import get_embeddings
import openai, os, tiktoken, backoff

openai.api_key = os.environ.get("OPENAI_API_KEY")
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
batch_size = 2000
max_tokens = 1000  # the maximum for text-embedding-ada-002 is 8191

df = pd.read_csv('20_newsgroup.csv')
print("Number of rows before null filtering:", len(df))
df = df[df['text'].isnull() == False]
encoding = tiktoken.get_encoding(embedding_encoding)

df["n_tokens"] = df.text.apply(lambda x: len(encoding.encode(x)))
print("Number of rows before token number filtering:", len(df))
df = df[df.n_tokens <= max_tokens]
print("Number of rows data used:", len(df))

Number of rows before null filtering: 11314
Number of rows before token number filtering: 11096
Number of rows data used: 10640


In [8]:
# 以下代码比较消耗 Token，你可以不运行
# @backoff.on_exception(backoff.expo, openai.error.RateLimitError)
# def get_embeddings_with_backoff(prompts, engine):
#     embeddings = []
#     for i in range(0, len(prompts), batch_size):
#         batch = prompts[i:i+batch_size]
#         embeddings += get_embeddings(list_of_text=batch, engine=engine)
#     return embeddings

# prompts = df.text.tolist()
# prompt_batches = [prompts[i:i+batch_size] for i in range(0, len(prompts), batch_size)]

# embeddings = []
# for batch in prompt_batches:
#     batch_embeddings = get_embeddings_with_backoff(prompts=batch, engine=embedding_model)
#     embeddings += batch_embeddings

# df["embedding"] = embeddings
# df.to_parquet("data/20_newsgroup_with_embedding.parquet", index=False)

In [29]:
import numpy as np
from sklearn.cluster import KMeans

embedding_df = pd.read_parquet("data/20_newsgroup_with_embedding.parquet")

matrix = np.vstack(embedding_df.embedding.values)
num_of_clusters = 20

kmeans = KMeans(n_clusters=num_of_clusters, init="k-means++", n_init=10, random_state=42)
kmeans.fit(matrix)
labels = kmeans.labels_
embedding_df["cluster"] = labels

In [30]:
embedding_df.head()

,text,target,title,n_tokens,embedding,cluster
0,I was wondering if anyone out there could enli...,7,rec.autos,121,"[-0.0391300804913044, 0.013502633199095726, -0...",5
1,\nIt depends on your priorities. A lot of peo...,7,rec.autos,108,"[-0.0011249205563217402, -0.00376517535187304,...",5
2,an excellent automatic can be found in the sub...,7,rec.autos,476,"[-0.018259447067975998, -0.008410007692873478,...",5
3,: Ford and his automobile. I need information...,7,rec.autos,86,"[-0.012589422054588795, 0.006539034191519022, ...",5
4,\nYo! Watch the attributions--I didn't say tha...,7,rec.autos,130,"[-0.0006192282889969647, -0.011226896196603775...",10


In [78]:

# 统计每个cluster的数量
new_df = embedding_df.groupby('cluster')['cluster'].count().reset_index(name='count')

# 统计这个cluster里最多的分类的数量
title_count = embedding_df.groupby(['cluster', 'title']).size().reset_index(name='title_count')
first_titles = title_count.groupby('cluster').apply(lambda x: x.nlargest(1, columns=['title_count']))
first_titles = first_titles.reset_index(drop=True)
new_df = pd.merge(new_df, first_titles[['cluster', 'title', 'title_count']], on='cluster', how='left')
new_df = new_df.rename(columns={'title': 'rank1', 'title_count': 'rank1_count'})

# 统计这个cluster里第二多的分类的数量
second_titles = title_count[~title_count['title'].isin(first_titles['title'])]
second_titles = second_titles.groupby('cluster').apply(lambda x: x.nlargest(1, columns=['title_count']))
second_titles = second_titles.reset_index(drop=True)
new_df = pd.merge(new_df, second_titles[['cluster', 'title', 'title_count']], on='cluster', how='left')
new_df = new_df.rename(columns={'title': 'rank2', 'title_count': 'rank2_count'})
new_df.fillna(0, inplace=True)
new_df['per_1'] = (new_df['rank1_count'] / new_df['count']).map(lambda x: '{:.2%}'.format(x))
new_df['per_1_2'] = ((new_df['rank1_count'] + new_df['rank2_count'])/ new_df['count']).map(lambda x: '{:.2%}'.format(x))

# 将缺失值替换为 0
# 输出结果
display(new_df)

,cluster,count,rank1,rank1_count,rank2,rank2_count,per_1,per_1_2
0,0,432,comp.windows.x,406,comp.sys.mac.hardware,1.0,93.98%,94.21%
1,1,418,sci.space,388,alt.atheism,2.0,92.82%,93.30%
2,2,1035,comp.sys.ibm.pc.hardware,396,comp.sys.mac.hardware,387.0,38.26%,75.65%
3,3,471,rec.sport.hockey,455,0,0.0,96.60%,96.60%
4,4,716,talk.politics.misc,270,alt.atheism,150.0,37.71%,58.66%
5,5,511,rec.autos,420,comp.sys.mac.hardware,6.0,82.19%,83.37%
6,6,870,rec.motorcycles,100,alt.atheism,73.0,11.49%,19.89%
7,7,570,comp.os.ms-windows.misc,338,comp.sys.mac.hardware,46.0,59.30%,67.37%
8,8,435,talk.politics.mideast,372,alt.atheism,23.0,85.52%,90.80%
9,9,84,comp.os.ms-windows.misc,8,comp.sys.mac.hardware,8.0,9.52%,19.05%


## 使用提示语对文本进行总结

In [94]:
items_per_cluster = 10
COMPLETIONS_MODEL = "text-davinci-003"

for i in range(num_of_clusters):
    cluster_name = new_df[new_df.cluster == i].iloc[0].rank1
    print(f"Cluster {i}, Rank 1: {cluster_name}, Theme:", end=" ")

    content = "\n".join(
        embedding_df[embedding_df.cluster == i].text.sample(items_per_cluster, random_state=42).values
    )
    response = openai.Completion.create(
        model=COMPLETIONS_MODEL,
        prompt=f'''我们想要给下面的内容，分组成有意义的类别，以便我们可以对其进行总结。请根据下面这些内容的共同点，总结一个50个字以内的新闻组的名称么？比如 “PC硬件”\n\n内容:\n"""\n{content}\n"""新闻组名称：''',
        temperature=0,
        max_tokens=100,
        top_p=1,
    )
    print(response["choices"][0]["text"].replace("\n", ""))


Cluster 0, Rank 1: comp.windows.x, Theme: Xlib编程
Cluster 1, Rank 1: sci.space, Theme: 太空技术与航空
Cluster 2, Rank 1: comp.sys.ibm.pc.hardware, Theme: PC硬件与系统
Cluster 3, Rank 1: rec.sport.hockey, Theme: 欧洲冰球vs北美冰球
Cluster 4, Rank 1: talk.politics.misc, Theme: 社会观点与自由
Cluster 5, Rank 1: rec.autos, Theme: 汽车硬件
Cluster 6, Rank 1: rec.motorcycles, Theme: 数学与文化冲击
Cluster 7, Rank 1: comp.os.ms-windows.misc, Theme: PC软件与硬件
Cluster 8, Rank 1: talk.politics.mideast, Theme: “穆斯林大屠杀”
Cluster 9, Rank 1: comp.os.ms-windows.misc, Theme: 科技产品"""
Cluster 10, Rank 1: talk.politics.guns, Theme: 枪支管制与安全
Cluster 11, Rank 1: comp.graphics, Theme: 计算机编程与硬件
Cluster 12, Rank 1: rec.motorcycles, Theme: 骑行安全与技巧
Cluster 13, Rank 1: soc.religion.christian, Theme: 宗教信仰与实践
Cluster 14, Rank 1: rec.sport.baseball, Theme: 棒球联盟
Cluster 15, Rank 1: misc.forsale, Theme: 购物优惠和出售
Cluster 16, Rank 1: sci.crypt, Theme: 关于加密政策的讨论
Cluster 17, Rank 1: sci.electronics, Theme: 电子设备技术
Cluster 18, Rank 1: sci.med, Theme: 药物和疾病
Cluster 1

In [101]:
items_per_cluster = 1
COMPLETIONS_MODEL = "text-davinci-003"

for i in range(num_of_clusters):
    cluster_name = new_df[new_df.cluster == i].iloc[0].rank1
    print(f"Cluster {i}, Rank 1: {cluster_name}, 抽样翻译:", end=" ")

    content = "\n".join(
        embedding_df[(embedding_df.cluster == i) & (embedding_df.n_tokens < 100)].text.sample(items_per_cluster, random_state=42).values
    )
    response = openai.Completion.create(
        model=COMPLETIONS_MODEL,
        prompt=f'''请把下面的内容翻译成中文\n\n内容:\n"""\n{content}\n"""翻译：''',
        temperature=0,
        max_tokens=500,
        top_p=1,
    )
    print(response["choices"][0]["text"].replace("\n", ""))

Cluster 0, Rank 1: comp.windows.x, 抽样翻译: 没有实际执行它？不知怎么回事，我的一个xterminal用户使得只要点击鼠标右键，就会自动杀死所有客户端-哦，我的：-(谢谢，Fish
Cluster 1, Rank 1: sci.space, 抽样翻译: 韦恩·马森和他的团伙在阿拉巴马州发生了什么？我还听说有一个未经证实的谣言，即航空大使们已经消失了。有其他人可以证实吗？
Cluster 2, Rank 1: comp.sys.ibm.pc.hardware, 抽样翻译: 我怀疑这不是一个特定于Quadra的问题。去年我不得不放弃我“古老”的Bernoulli 20（每个磁带的价格大约是90美元，使整个事情的价值超过我的整个电脑;）。Ocean Microsystems的技术支持人员建议可以使用一些第三方驱动程序来解决这个问题 - 在我的情况下，磁带无法格式化/挂载/分区用于A / UX。
Cluster 3, Rank 1: rec.sport.hockey, 抽样翻译: 我相信那是4-1。罗德·布林道·阿莫尔在第三节19.59时分攻入一球。
Cluster 4, Rank 1: talk.politics.misc, 抽样翻译: 为了确保每个人都清楚：“它从未有过”是指“保护”，而不是“未能保护”；即，在我的一生中，我从未见过美国政府始终保护美国公民的利益，除非是意外。
Cluster 5, Rank 1: rec.autos, 抽样翻译: 噢，来吧，傻瓜，你要做的就是在你的引擎罩上割一个洞，然后把一个管子放进去，这样你就可以把机油倒进去了。你觉得那些热门车上的大空气进气装置是干什么的？它们只是为了外观，没有人知道，它们提供了进入机油填充孔的途径。
Cluster 6, Rank 1: rec.motorcycles, 抽样翻译: 你真是个失败者
Cluster 7, Rank 1: comp.os.ms-windows.misc, 抽样翻译: 偶尔你需要为表现良好的东西说句好话。我的东西桥3401没有任何问题。它在DOS和OS/2上运行得很好。对于OS/2，你不需要加载任何特殊的驱动程序。安装会检测到它是一个东西桥驱动器，然后就完成了。顺便说一句，它也很快！
Cluster 8, Rank 1: talk.politics.m

从输出的结果来看，我们选取的内容和AI总结的标题的确八九不离十。不知道这个过程，有没有重新让你回忆起Completion这个接口的强大之处。我们只要给出合适的指令，它就能按照我们的心意给出合理的结果。


## 利用文本摘要支持无限轮数的聊天

In [106]:
history = """User : 你是谁？
Assistant : 我是一个人工智能语言模型，可以回答各种问题，包括做菜的问题。

User : 请问鱼香肉丝怎么做？
Assistant : 鱼香肉丝的做法：

1. 猪里脊肉切丝，加入盐、淀粉、料酒搅拌均匀，腌制10分钟
2. 胡萝卜、青椒、木耳切丝备用
3. 热锅凉油，放入蒜末、姜末煸炒出香味，加入肉丝翻炒至变色
4. 加入胡萝卜、青椒、木耳翻炒，加入酱油、糖、醋、盐、味精、水淀粉搅拌均匀，倒入锅中翻炒均匀即可。

提示：鱼香肉丝的关键是调料的配比，酱油、糖、醋、盐、味精的比例要适合自己的口味。

User : 那蚝油牛肉呢？
Assistant : 蚝油牛肉的做法：

1. 将牛肉切成薄片，加入盐、淀粉、料酒腌制10分钟
2. 葱姜切末，蒜切片，青红椒切丝备用
3. 热锅凉油，放入葱姜末、蒜片煸炒出香味，加入牛肉翻炒至变色
4. 加入蚝油、酱油、糖、水淀粉调味，加入青红椒翻炒均匀即可。

提示：蚝油牛肉的关键是牛肉要切薄，翻炒时火候要快，保证牛肉口感鲜嫩。调味时，蚝油和酱油的比例也要适合自己的口味。
"""

def summarize(text, max_tokens=200):
    response = openai.Completion.create(
        model=COMPLETIONS_MODEL,
        prompt=text + "\n\n请总结一下上面User和Assistant聊了些什么：\n",
        max_tokens=max_tokens,
    )
    return response["choices"][0]["text"]

summarized = summarize(history)
print(summarized)


User和Assistant聊了鱼香肉丝和蚝油牛肉的制作方法。User问了Assistant两个关于如何做鱼香肉丝和蚝油牛肉的问题，Assistant给出了回答并介绍了每道菜的具体制作方法，同时也提示了调料的配比和牛肉制作时要注意的细节。


In [107]:

class Conversation:
    def __init__(self, prompt, num_of_round):
        self.prompt = prompt
        self.num_of_round = num_of_round
        self.messages = []
        self.messages.append({"role": "system", "content": self.prompt})

    def ask(self, question):
        try:
            self.messages.append( {"role": "user", "content": question})
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=self.messages,
                temperature=0.5,
                max_tokens=2048,
                top_p=1,
            )
        except Exception as e:
            print(e)
            return e

        message = response["choices"][0]["message"]["content"]
        self.messages.append({"role": "assistant", "content": message})
        
        if len(self.messages) > self.num_of_round*2 + 1:
            del self.messages[1:3]
        return message

In [108]:
prompt = summarized + "\n\n请你根据已经聊了的内容，继续对话："
conversation = Conversation(prompt, 5)

question = "那宫保鸡丁呢？"
answer = conversation.ask(question)
print("User : %s" % question)
print("Assistant : %s\n" % answer)

User : 那宫保鸡丁呢？
Assistant : 宫保鸡丁的制作方法也比较简单。首先，将鸡肉切成小丁状，用料酒、盐、生抽腌制一下。然后将青椒、红椒、葱姜蒜切成丁状备用。接着，将花生米炒香备用。

热锅凉油，油温七成热时放入鸡丁煸炒至变色，捞出备用。再将葱姜蒜爆香，加入青红椒丁翻炒一下，然后加入鸡丁，翻炒均匀。最后加入适量的糖、盐、醋、生抽、老抽、料酒、水淀粉炒匀，最后加入炒香的花生米即可。

需要注意的是，炒鸡丁的时候要用大火，这样鸡肉会更嫩。另外，调料的配比也很关键，需要根据个人口味适量调整。



In [109]:
conversation = Conversation("请你根据已经聊了的内容，继续对话：", 5)

question = "那宫保鸡丁呢？"
answer = conversation.ask(question)
print("User : %s" % question)
print("Assistant : %s\n" % answer)

User : 那宫保鸡丁呢？
Assistant : 宫保鸡丁是一道非常有名的川菜，口感麻辣鲜香，非常美味。你喜欢吃辣的食物吗？

